In [1]:
import numpy as np
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch

In [2]:
# X, y = make_classification(n_samples=99999, n_informative = 2, n_classes=3, n_features=4)
X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

y_train = y_train
y_test = y_test

In [3]:
from simple_model import ClassifierModel, DyadRankingModel

clf = ClassifierModel(output_dim=y.max()+1)
rnk = DyadRankingModel(num_classes=y.max()+1)

clf.fit(X_train,y_train, batch_size=1)
rnk.fit(X_train,y_train, batch_size=1)

In [4]:
from sklearn.metrics import accuracy_score
y_pred_clf = clf.predict(X_test)
y_pred_rnk = rnk.predict_classes(X_test)
print(accuracy_score(y_test, y_pred_clf))
print(accuracy_score(y_test, y_pred_rnk))

0.96
0.98
